In [1]:
import dask
import logging
from datetime import datetime
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=10)

from grizly.scheduling.registry import Trigger, Job

2020-10-22 21:09:04,277 - grizly.config - DEBUG - Config data loaded from /home/analyst/.grizly/config.json.


## Job

In [2]:
@dask.delayed
def print_time():
    print(f"Hello, it's {datetime.now().__str__()}")
    
time_log = print_time()

@dask.delayed
def add(x, y):
    return x + y

_sum = add(1, 2)

In [3]:
job1 = Job("test_print")
job1.register(tasks=[time_log], 
              crons="* * * * *", 
              if_exists="replace")

job2 = Job("test_sum")
job2.register(tasks=[_sum], 
              upstream="test_print", 
              if_exists="replace")

2020-10-22 21:09:04,873 - grizly.config - DEBUG - Config data loaded from /home/analyst/.grizly/config.json.
2020-10-22 21:09:06,979 - grizly.scheduling.registry - DEBUG - Job(name='test_print') with cron '* * * * *' has been added to rq sheduler with id c66ce92c-d51a-4466-bc69-972e3826374f
2020-10-22 21:09:06,980 - grizly.scheduling.registry - DEBUG - Job(name='test_print') has been added to the rq scheduler
2020-10-22 21:09:07,628 - grizly.scheduling.registry - INFO - Job(name='test_print') successfully registered
2020-10-22 21:09:07,631 - grizly.config - DEBUG - Config data loaded from /home/analyst/.grizly/config.json.
2020-10-22 21:09:09,872 - grizly.scheduling.registry - INFO - Adding ['test_sum'] to Job(name='test_print').downstream...
2020-10-22 21:09:10,979 - grizly.scheduling.registry - INFO - Job(name='test_sum') successfully registered


Job(name='test_sum')

In [4]:
job1.info()

name: test_print
owner: None
description: None
timeout: 3600
created_at: 2020-10-22 21:09:05.382143+00:00
crons: ['* * * * *']
downstream: [Job(name='test_sum')]
upstream: []
triggers: []


In [5]:
job1.runs

[]

In [6]:
job2.info()

name: test_sum
owner: None
description: None
timeout: 3600
created_at: 2020-10-22 21:09:08.277146+00:00
crons: []
downstream: []
upstream: [Job(name='test_print')]
triggers: []


In [7]:
job2.runs

[]

In [8]:
job1.submit(to_dask=False)

2020-10-22 21:09:18,295 - grizly.scheduling.registry - INFO - Submitting job test_print...


Hello, it's 2020-10-22 21:09:19.960816


2020-10-22 21:09:21,757 - grizly.scheduling.registry - INFO - Job Job(name='test_print') finished with status success
2020-10-22 21:09:23,673 - grizly.scheduling.registry - INFO - Enqueueing Job(name='test_print').downstream...
2020-10-22 21:09:26,241 - grizly.scheduling.registry - DEBUG - Job(name='test_sum') has been added to rq scheduler with id 09c7f074-40a4-4659-8952-8dad97eddbda
2020-10-22 21:09:26,242 - grizly.scheduling.registry - INFO - Job(name='test_sum') has been enqueued


[None]

In [9]:
job1.last_run.info()

id: 1
name: None
created_at: 2020-10-22 21:09:18.621996+00:00
finished_at: 2020-10-22 21:09:21.758108+00:00
duration: 2
status: success
error: None
result: [None]


In [10]:
job1.unregister(remove_job_runs=True)

2020-10-22 21:09:29,331 - grizly.scheduling.registry - DEBUG - Rq job c66ce92c-d51a-4466-bc69-972e3826374f removed from the rq scheduler
2020-10-22 21:09:29,332 - grizly.scheduling.registry - DEBUG - Job(name='test_print') has been removed from the rq scheduler
2020-10-22 21:09:31,080 - grizly.scheduling.registry - INFO - Removing ['test_print'] from Job(name='test_sum').upstream...
2020-10-22 21:09:32,375 - grizly.scheduling.registry - INFO - Removing ['test_sum'] from Job(name='test_print').downstream...
2020-10-22 21:09:35,445 - grizly.scheduling.registry - INFO - Job(name='test_print')'s runs have been removed from registry
2020-10-22 21:09:35,761 - grizly.scheduling.registry - INFO - Job(name='test_print') successfully removed from registry


In [11]:
job2.unregister()

2020-10-22 21:09:37,336 - grizly.scheduling.registry - DEBUG - Rq job 09c7f074-40a4-4659-8952-8dad97eddbda removed from the rq scheduler
2020-10-22 21:09:37,337 - grizly.scheduling.registry - DEBUG - Job(name='test_sum') has been removed from the rq scheduler
2020-10-22 21:09:38,939 - grizly.scheduling.registry - INFO - Job(name='test_sum') successfully removed from registry
